# Ejercicios Clase: Álgebra en Ciencia de Datos
### Estudiante: Edgard Iglesias Rubio

## Unidad 1
### Análisis de tópicos

1. Realice entre 5 y 10 consultas sobre temas distintos en el portal CREA | Real Academia Española, cada consulta debe retornar al menos 50 concordancias. Construya una matriz de frecuencia de términos a partir de esas consultas. Es necesario guardar tanto el vocabulario como la matríz. 

In [1]:
!pip install nltk
!pip install spacy
!pip install unidecode
!python -m spacy download es_core_news_md

     --------------------------------------- 42.3/42.3 MB 13.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [2]:
#imports
import os
import pandas as pd
import unidecode
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from random import shuffle
import scipy as sp
from sklearn.decomposition import NMF

#nlp
from string import punctuation
import spacy
nlp = spacy.load('es_core_news_md')
from spacy.lang.es import stop_words

Cargando archivos descargados desde CREA | Rae hacia un dataframe (6) archivos:

In [3]:
ruta_files='filesRAE/'
df=pd.DataFrame()
for f in os.listdir(ruta_files):
    dfaux=pd.read_csv(ruta_files+f,sep='\t', skiprows=8)
    df=pd.concat([df,dfaux])

In [4]:
df.head()

,BIBLIOGRAFÍA,AUTOR,TÍTULO,FECHA,CRITERIO,BLOQUE,MEDIO,SOPORTE,TEMA,PAÍS,ZONA,TIPOLOGÍA,NOTAS,CONCORDANCIA
0,«El CIS había anunciado severos controles para...,NaN,El País,1976,Primera edición,No ficción,Escrito,Prensa,"Actualidad, ocio y vida cotidiana",España,NaN,NaN,Mayúsculas no acentuadas en el original,Automovilismo
1,«Campeón del mundo de fórmula 1». El País. Mad...,NaN,El País,1976,Primera edición,No ficción,Escrito,Prensa,"Actualidad, ocio y vida cotidiana",España,NaN,NaN,Mayúsculas no acentuadas en el original,. Los comentarios que hagan algunas revistas a...
2,«Campeón del mundo de fórmula 1». El País. Mad...,NaN,El País,1976,Primera edición,No ficción,Escrito,Prensa,"Actualidad, ocio y vida cotidiana",España,NaN,NaN,Mayúsculas no acentuadas en el original,— ¿Sus ídolos del automovilismo?
3,"«El Wolf WRL, nuevo bólido de fórmula-1». El P...",NaN,El País,1976,Primera edición,No ficción,Escrito,Prensa,"Actualidad, ocio y vida cotidiana",España,NaN,NaN,Mayúsculas no acentuadas en el original,Automovilismo
4,«Ocho elecciones federativas para febrero y ma...,NaN,El País,1977,Primera edición,No ficción,Escrito,Prensa,"Actualidad, ocio y vida cotidiana",España,NaN,NaN,Mayúsculas no acentuadas en el original,Durante el mes de febrero las Federaciones Nac...


In [5]:
#Creando listas de stop words y de puntuaciones para depurar tokens:
stop_words_list = [unidecode.unidecode(word) for word in stop_words.STOP_WORDS]
punctuations = list(punctuation)

In [6]:
#Funcion que dado un texto retorna un texto tokenizado
def tokenizador(text):
    text = unidecode.unidecode(text)
    tokens = nlp(text)
    tokens = [token.lower_ for token in tokens if token.lower_ not in stop_words_list and token.pos_!='PUNCT' and token.is_alpha]
    tokens = [re.sub(r'[^A-Za-z0-9]+','',w) for w in tokens]
    tokens = [token for token in tokens if token not in punctuations]
    
    return ' '.join(tokens)

In [7]:
#aplicando tokenizador:
df['tokens'] = df.apply(lambda row: tokenizador(row['CONCORDANCIA']), axis=1)

In [8]:
df.head()

,BIBLIOGRAFÍA,AUTOR,TÍTULO,FECHA,CRITERIO,BLOQUE,MEDIO,SOPORTE,TEMA,PAÍS,ZONA,TIPOLOGÍA,NOTAS,CONCORDANCIA,tokens
0,«El CIS había anunciado severos controles para...,NaN,El País,1976,Primera edición,No ficción,Escrito,Prensa,"Actualidad, ocio y vida cotidiana",España,NaN,NaN,Mayúsculas no acentuadas en el original,Automovilismo,automovilismo
1,«Campeón del mundo de fórmula 1». El País. Mad...,NaN,El País,1976,Primera edición,No ficción,Escrito,Prensa,"Actualidad, ocio y vida cotidiana",España,NaN,NaN,Mayúsculas no acentuadas en el original,. Los comentarios que hagan algunas revistas a...,comentarios hagan revistas acerca persona inte...
2,«Campeón del mundo de fórmula 1». El País. Mad...,NaN,El País,1976,Primera edición,No ficción,Escrito,Prensa,"Actualidad, ocio y vida cotidiana",España,NaN,NaN,Mayúsculas no acentuadas en el original,— ¿Sus ídolos del automovilismo?,idolos automovilismo
3,"«El Wolf WRL, nuevo bólido de fórmula-1». El P...",NaN,El País,1976,Primera edición,No ficción,Escrito,Prensa,"Actualidad, ocio y vida cotidiana",España,NaN,NaN,Mayúsculas no acentuadas en el original,Automovilismo,automovilismo
4,«Ocho elecciones federativas para febrero y ma...,NaN,El País,1977,Primera edición,No ficción,Escrito,Prensa,"Actualidad, ocio y vida cotidiana",España,NaN,NaN,Mayúsculas no acentuadas en el original,Durante el mes de febrero las Federaciones Nac...,mes febrero federaciones nacionales balonmano ...


In [9]:
#generando BoW
vectorizer = CountVectorizer(min_df=2)
bow = vectorizer.fit_transform(df['tokens'])
print("Número de documentos: {docs}, Tamaño Vocabulario: {voc}".format(docs=bow.shape[0], voc=bow.shape[1]))

Número de documentos: 2592, Tamaño Vocabulario: 8281


In [10]:
#Intercambiando filas
indices = np.arange(bow.shape[0]) # obtener los indices de las columnas
shuffle(indices) #mover aleatoreamente
bow_shuffled = bow[list(indices)]

In [11]:
#Bow es una matriz dispersa la cual guardaremos en formato npz
sp.sparse.save_npz('bow_sparse.npz', bow_shuffled)

In [12]:
#El vocabulario es una lista la cual guardaremos en formato npy
vocabulary = vectorizer.get_feature_names()
np.save('vocabularioBoW',vocabulary, allow_pickle=True)

2. Realice un análisis de tópicos usando una factorización no negativa (NMF) de la matriz construida en el punto anterior, el análisis busca recuperar los términos usados para la consulta y debe poder replicarse con la matriz y vocabulario generados por otra persona.

Si el bow esta en formato npz y el vocabulario en formato npy, se podran cargar los archivos de otra persona

In [13]:
#cargando bow y vocabulario

bow = sp.sparse.load_npz('bow_sparse.npz')
voc = np.load('vocabularioBoW.npy', allow_pickle=True)

In [14]:
bow

<2592x8281 sparse matrix of type '<class 'numpy.int64'>'
	with 52055 stored elements in Compressed Sparse Row format>

In [15]:
voc

array(['ab', 'abajo', 'abandonada', ..., 'zumbido', 'zurich', 'zurrido'],
      dtype='<U18')

Extracción de Tópicos a través de NMF, siguiendo esta estratégia:

![NMF](img/NMF.png)

In [16]:
# Probando para k=5
k = 5 # numero de topicos
bowT=bow.transpose()
nmf_model = NMF(n_components=k, init='random', random_state=42)  # Initialize NMF model
W = nmf_model.fit_transform(bowT)  # Matrix W
H = nmf_model.components_  # Matrix H

In [17]:
#funcion para identificar el top de tokens por topicos de la matriz W
def get_topics(df_W,vocBoWLoad,top_n):
  for i in df_W.columns:
    index_list=df_W[i].nlargest(top_n).index.to_list()
    tokens = [vocBoWLoad[j] for j in index_list]
    print('Topic {i}: {tokens}'.format(i=i,tokens=tokens))

In [18]:
get_topics(pd.DataFrame(W),voc,5)

Topic 0: ['motor', 'coche', 'marcha', 'automovil', 'ruido']
Topic 1: ['neumaticos', 'vehiculos', 'coche', 'tipo', 'carrera']
Topic 2: ['escuderia', 'piloto', 'schumacher', 'mundial', 'ferrari']
Topic 3: ['automovilismo', 'millones', 'federacion', 'pesetas', 'presidente']
Topic 4: ['formula', 'premio', 'mundo', 'circuito', 'ano']


El top 5 de topicos buscados es: motor, automovilismo, formula, escuderia, neumaticos. Que corresponen al tema principal que busqué sobre Formula 1